In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import pandas as pd
import requests
import json

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess


class Chewy_adult(scrapy.Spider):
    name = "Chewy_adult"
    start_urls = ['https://www.chewy.com/b/adult-11098']

    def parse(self, response):
        for products in response.css('div.kib-product-card__content'):
            yield {
                  'name':  products.css('a.kib-product-title *::text').get(),
             'food_type':  products.css('a.kib-product-title *::text').re_first(r"^\s{1}[(\w\s)-|&|']+"),
                 'price':  products.css('div.kib-product-price--md::text').get(),
                'rating':  products.css('div.kib-product-rating__label::text').get(),
             'no_review':  products.css('span.kib-product-rating__count::text').get(),
                 'link' : products.css('a.kib-product-title::attr(href)').get(),
            }
            
        next_page= response.css('a.kib-pagination-new-item--next').attrib['href']
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)

process = CrawlerProcess(settings={
    "FEEDS": {
        "reg_adults.csv": {"format": "csv"},
    },
})

process.crawl(Chewy_adult)
process.start()


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
adults = pd.read_csv("reg_adults.csv")
adults = adults.dropna().reset_index(drop=True)
adults

In [ ]:
#Remove grain-free and bundled products 
adults = adults.loc[~adults['food_type'].str.contains("Grain-free|Grain-Free|Bundle")]

In [ ]:
adults['price'] = adults['price'].str.replace('$', '')
adults['no_review'] = adults['no_review'].str.replace(',', '')

In [ ]:
adults['price'] = adults['price'].astype('float')

adults['no_review'] = adults['no_review'].astype('int')

In [ ]:
pattern = r'(\d\.\d*)'
adults['rating'] = adults['rating'].str.extract(pattern)

adults['rating'] = adults['rating'].astype('float')

In [ ]:
adults_100=adults.loc[adults['no_review']>100].reset_index(drop=True)
adults_100

In [ ]:
adult_100_link = adults_100.link.str.extract(r'dp\/(\d+)', expand = True)
adult_link = adult_100_link[0].values.tolist()
len(adult_link)

In [ ]:
URL = 'https://www.chewy.com/api/pdp-graphql/graphql'
headers = {
    'user-agent': 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36'
}

def get_params(product_id,after):
    params = {
    'operationName': 'Reviews',
    'variables': '{"id":"'+product_id+'","first":100,"sort":"MOST_RELEVANT","after":"'+after+'"}',
    'extensions': '{"persistedQuery":{"version":1,"sha256Hash":"f1ad95c550af020ebbc5c6da08fd478b1cf25b5e911cba4710d773b84e063730"}}'
}
    return params

In [ ]:
 def get_reviews(product_id):
    
        reviews = []
        hasNextPage = True
        after = "YXJyYXljb25uZWN0aW9uOjk="

        while hasNextPage:
            params = get_params(product_id,after)
            response = requests.get(URL,
                                params = params,
                                headers = headers)
            #return response
            
            after = (response.json()['data']['product']['reviews']['pageInfo']['endCursor'])
            hasNextPage = (response.json()['data']['product']['reviews']['pageInfo']['hasNextPage'])
            rev = (response.json()['data']['product']['reviews']['edges'])
            reviews.extend(rev)

            #print("Getting next page")

        filepath = f"data/reviews_adult/reviews_{product_id}.json"
        with open(filepath,"w") as fi:
            json.dump(reviews, fi)
            #reviews.append(ids)

In [ ]:
product_id= plz

for product_id in plz:
    
   get_reviews(product_id)

In [ ]:
adults_100.describe()